# 🌐 Conectores Avanzados: REST, GraphQL y SFTP

Objetivo: dominar integración robusta con APIs REST/GraphQL y transferencia de archivos por SFTP, aplicando autenticación (API Key/OAuth2), paginación, rate limiting, reintentos exponenciales, validación de esquemas y almacenamiento seguro.

- Duración: 90–120 min
- Dificultad: Media
- Prerrequisitos: Python, requests/httpx, fundamentos de redes

## 0. Dependencias y configuración

- REST: `requests` (sync) o `httpx` (async).
- GraphQL: se puede usar `requests` o `gql` (opcional).
- SFTP: `paramiko` (opcional, no incluido por defecto).
- Variables de entorno sugeridas: `API_BASE_URL`, `API_KEY`, `OAUTH_TOKEN_URL`, `OAUTH_CLIENT_ID`, `OAUTH_CLIENT_SECRET`, `SFTP_HOST`, `SFTP_USER`.

## 1. REST con requests: paginación, backoff y validación

In [ ]:
import os, time, math
import requests
from typing import Dict, Any, List

BASE_URL = os.getenv('API_BASE_URL', 'https://api.publicapis.org')
API_KEY = os.getenv('API_KEY')  # si aplica

def get_with_backoff(url: str, headers: Dict[str,str]=None, params: Dict[str,Any]=None, max_retries: int=5):
    for i in range(max_retries):
        resp = requests.get(url, headers=headers, params=params, timeout=30)
        if resp.status_code == 429:  # rate limit
            wait = 2 ** i
            time.sleep(wait)
            continue
        resp.raise_for_status()
        return resp.json()
    raise RuntimeError('Max retries exceeded')

data = get_with_backoff(f'{BASE_URL}/entries')
len(data.get('entries', []))

### 1.1 Paginación cursor/offset y almacenamiento incremental

In [ ]:
def fetch_paginated(base_url: str, page_param='page', per_page=50, limit=3):
    all_items: List[Dict[str,Any]] = []
    for p in range(1, limit+1):
        payload = get_with_backoff(base_url, params={page_param: p, 'per_page': per_page})
        items = payload.get('data') or payload.get('entries') or []
        all_items.extend(items)
        if not items:
            break
    return all_items

items = fetch_paginated(f'{BASE_URL}/entries', page_param='page', per_page=20, limit=2)
len(items)

### 1.2 Validación de esquema con Cerberus/Pandera

In [ ]:
from cerberus import Validator
schema = {
  'API': {'type':'string', 'required': True},
  'Description': {'type':'string', 'nullable': True},
  'HTTPS': {'type':'boolean'},
}
v = Validator(schema, allow_unknown=True)
valid_count = sum(1 for it in items if v.validate(it))
valid_count, len(items)

## 2. OAuth2 Client Credentials (httpx) [opcional]

In [ ]:
import httpx
async def fetch_oauth_token():
    token_url = os.getenv('OAUTH_TOKEN_URL')
    if not token_url:
        return None
    data = {
      'grant_type':'client_credentials',
      'client_id': os.getenv('OAUTH_CLIENT_ID'),
      'client_secret': os.getenv('OAUTH_CLIENT_SECRET'),
      'scope': os.getenv('OAUTH_SCOPE','')
    }
    async with httpx.AsyncClient(timeout=30) as client:
        r = await client.post(token_url, data=data)
        r.raise_for_status()
        return r.json().get('access_token')

# Uso: token = asyncio.run(fetch_oauth_token())

## 3. GraphQL: consultas y paginación

In [ ]:
GQL_URL = os.getenv('GQL_URL', 'https://countries.trevorblades.com/')
query = 

,
,
,
,

:
,
:{},
:[
4
]},
:
,
:{},
:[
22
,
,
,
,

:
,
:{},
:[
5
]},
:
,
:{},
:[
,
,
,

: {
: {
: 
3
, 
: 
, 
: 
}, 
: {
: 
, 
: 
3.8
}},
: 4,
: 4